## N-Queens Problem

*Given a NxN chessboard, can you place N queens such that they do not kill one another?*

We will solve this as a constraint satisfaction problem, using the **minimum conflicts approach**. The idea is to repeatedly try to minimize the conflicts among the variables by picking a single random conflicted variable and changing its value.

In [1]:
from collections import defaultdict
import itertools

import numpy as np

In [2]:
class NQueens:
    """Represents the N Queens problem as a constraint satisfaction problem.
    
    Data structure: dict (variable, value) format.
    Variable: the row index, e.g. 0-7 in case of 8-queens
    Value: the col index, again 0-7 (we change this)
    """
    
    def __init__(self, N=8, assignment='random'):
        """Initializes a NxN chessboard. Only stores positions of queens.
        
        Two methods to initialize the problem: greedy and random. Greedy generally
        helps find a solution quicker than random initialization. Greedy initialization
        selects the first value that is compatible with variables initialized so far.
        """
        self.N = N
        cols = list(range(N))
        np.random.shuffle(cols)
        self.assignment = {i: j for i, j in zip(range(N), cols)}
        # TODO: greedy initialization
        self.cache = defaultdict(list)
        for k, v in self.assignment.items():
            self.cache[k].append(v)
    
    def is_conflict(self, x1, x2, assignment):
        """Check if two variables (representing row numbers) conflict."""
        y1 = assignment[x1]
        y2 = assignment[x2]
        return x1 == x2 or y1 == y2 or (abs(x1 - x2) == abs(y1 - y2))
    
    def is_solution(self, assignment):
        """Check if the current assignment is a valid solution."""
        return len(assignment) == self.N and len(self.conflicted_vars(assignment)) == 0
        
    def n_conflicts(self, var, assignment):
        """Counts the number of conflicts for a particular variable (row index)."""
        # check all combinations
        return sum([self.is_conflict(var, el, assignment)
                    for el in assignment if el != var])
    
    def conflicted_vars(self, assignment):
        """Return the list of variables that are conflicted."""
        return [v for v in assignment if self.n_conflicts(v, assignment) > 0]
    
    def record(self, var, val):
        """Track the changes we have tried."""
        self.cache[var].append(val)
    
    def min_conflicts_val(self, var, assignment):
        """Return the value for a variable that minimizes the number of conflicts."""
        m = self.n_conflicts(var, assignment)
        val = assignment[var]
        vals_to_try = [x for x in range(self.N) if x not in self.cache[var]]
        if len(vals_to_try) == 0:
            vals_to_try = list(range(self.N))
        for j in vals_to_try:
            if j == val:
                continue
            assignment[var] = j
            tmp = self.n_conflicts(var, assignment)
            if tmp <= m:
                m = tmp
                val = j
        return val
    
    def min_conflicts(self, max_steps=1000):
        """Min conflicts approach to solving this problem."""
        current = self.assignment
        for n_iter in range(max_steps):
            if self.is_solution(current):
                print(f'Found solution in {n_iter} iterations')
                return current
            var = np.random.choice(self.conflicted_vars(current))
            val = self.min_conflicts_val(var, current)
            current[var] = val
            self.record(var, val)  # to avoid running into a deadlock like situation
        return f'Failed after {max_steps} tries. Try with more iterations or a different initial state.'

The min conflicts approach is very effective in solving this type of problem because of the dense nature of the state space.

**More things to try:**
- Try larger values of N (implement in Java or Cython)
- Compare (both memory and execution time) with solving the same problem using search methods and backtracking.
- Random vs. greedy initialization: does greedy initialization help arriving at the solution quicker?

In [3]:
n_queens = NQueens()
tmp = n_queens.min_conflicts()

Found solution in 12 iterations


### Intelligent Backtracking Search

In [4]:
# TODO: test backtracking and min conflicts, create just one class after testing

class NQueensSearch:
    """Represents the NQueens problem and solves it using backtracking search."""
    
    def __init__(self, N=8):
        self.N = N
        # start with a partial assignment
        self.assignment = {0: np.random.randint(0, 8)}
        
    def is_complete(self, assignment):
        """Check if the current assignment is complete."""
        return len(assignment) == self.N
    
    def is_solution(self, assignment):
        """Checks if the current assignment is a valid solution."""
        if assignment == 'failure':
            return False
        # check for conflicts among all combinations of keys
        for k1, k2 in itertools.combinations(assignment.keys(), 2):
            if self.is_conflict(k1, k2, assignment[k1], assignment[k2]):
                return False
        return True
    
    def is_conflict(self, x1, x2, y1, y2):
        """Check if two variables (representing row numbers) conflict."""
        return x1 == x2 or y1 == y2 or abs(x1 - x2) == abs(y1 - y2)

    def remaining_values(self, var, assignment):
        """Calculates the number of legal values remaining for a variable."""
        # idea is to check a particular value with all other variables in the assignment
        values = list(range(self.N))
        legal = []
        for val in values:
            if self.is_consistent(var, val, assignment):
                legal.append(val)
        return legal
    
    def select_unassigned_variable(self, assignment):
        """Selects a unassigned variable using MRV heuristic."""
        unassigned = [(var, self.remaining_values(var, assignment))
                      for var in range(self.N) if var not in assignment]  # (var, n_legal)
        if unassigned:  # minimum remaining values heuristic
            return sorted(unassigned, key=lambda x: len(x[1]))[0]
        else:
            return None
        
    def number_of_choices_ruled_out(self, var, val, neighbor, assignment):
        """If var takes the value val, how many choices are ruled out for the neighboring variable."""
        # calculate number of choices with assignment
        legal_values = self.remaining_values(neighbor, assignment)
        # compute how many of those are in conflict with var:val
        n_choices_ruled_out = 0
        for v in legal_values:
            if self.is_conflict(var, neighbor, val, v):
                n_choices_ruled_out += 1
        return n_choices_ruled_out       
    
    def order_domain_values(self, var, values, assignment):
        """Choose the value that rules out the fewest choices for neighboring variables. This
        is called the least-constraining-value heuristic."""
        neighbors = [var-1, var+1]   # recall, row index is variable
        ruled_out = self.N * 2  # assign max possible
        result = []   # contain values in lcv order
        for val in values:
            n_choices_ruled_out = 0
            for neighbor in neighbors:
                if neighbor not in assignment:
                    n_choices_ruled_out += self.number_of_choices_ruled_out(var, val, neighbor, assignment)
            result.append((val, n_choices_ruled_out))
        result = sorted(result, key=lambda x: x[1])
        if result:
            return [x[0] for x in result]
        else:
            return []
    
    def inference(self, var, value):
        """Inference on constraints such as arc-consistency or path-consistency."""
        # TODO
        return {}
        
    def is_consistent(self, var, value, assignment):
        """Checks if the var:value is consistent with this assignment, i.e. no conflicts."""
        for x, y in assignment.items():
            if self.is_conflict(x, var, y, value):
                return False
        return True
        
    def backtrack(self, assignment):
        """Recursive backtracking function to search for valid solutions.
        
        Args:
            assignment: a partial or complete assignment
            
        Returns:
            solution, if found or failure, if no solution found
        """
        if self.is_complete(assignment):
            return assignment
        var, values = self.select_unassigned_variable(assignment)
        for val in self.order_domain_values(var, values, assignment):
            if self.is_consistent(var, val, assignment):
                assignment[var] = val
                inferences = self.inference(var, val)  # partial assignment
                if inferences != 'failure':
                    assignment = {**assignment, **inferences}
                    result = self.backtrack(assignment)
                    if result != 'failure':
                        return result
            # remove inferences and var from assignment
            assignment.pop(var, None)
            for k in inferences:
                assignment.pop(k, None)
        return 'failure'

In [5]:
nqueens = NQueensSearch(16)

In [6]:
solution = nqueens.backtrack(nqueens.assignment)
solution

{0: 4,
 1: 0,
 2: 3,
 7: 2,
 10: 7,
 6: 8,
 4: 9,
 3: 12,
 9: 1,
 8: 15,
 5: 13,
 12: 10,
 13: 6,
 14: 14,
 11: 5,
 15: 11}